In [1]:
import os
import time
print(os.environ['LSST_DDS_DOMAIN'])
os.environ['LSST_DDS_DOMAIN'] = 'auxtelpath'
print(os.environ['LSST_DDS_DOMAIN'])

auxtelpath
auxtelpath


In [2]:
from salpytools import salpylib 

Begin subscribing to events and telemetry
The events/telemetry are determined from looking the XML (from the stash repository)

In [3]:
listener = salpylib.DDSSubcriber(Device = "atMonochromator", 
                                 topic='SummaryState',
                                 Stype='Event')
heartbeat = salpylib.DDSSubcriber(Device = "atMonochromator", 
                                 topic='Heartbeat',
                                 Stype='Event')
rejected = salpylib.DDSSubcriber(Device = "atMonochromator", 
                                 topic='RejectedCommand',
                                 Stype='Event')
looptime = salpylib.DDSSubcriber(Device = "atMonochromator", 
                                 topic='LoopTime_ms',
                                 Stype='Telemetry')
grating = salpylib.DDSSubcriber(Device = "atMonochromator", 
                                 topic='SelectedGrating',
                                 Stype='Event')
detailed_state = salpylib.DDSSubcriber(Device = "atMonochromator", 
                                 topic='DetailedState',
                                 Stype='Event')

AttributeError: module 'salpytools.salpylib' has no attribute 'DDSSubcriber'

Now start listening

In [10]:
listener.start()
heartbeat.start()
rejected.start()
looptime.start()
grating.start()
detailed_state.start()

No event has been published since we started listening. This means that we do not know what grating is installed. Need a fix such that this is remedied (meaning need to know if it is alive, and then what grating is in)

In [11]:
print(listener.getCurrent().summaryState)
print(heartbeat.getCurrent().heartbeat)
print(looptime.getCurrent().loopTime_ms)
print(grating.getCurrent().gratingType)
print(detailed_state.getCurrent().detailedState)

[2018-07-11 14:47:33] [INFO] WARNING: No value received for: 'SummaryState' yet, sending empty object anyway
[2018-07-11 14:47:33] [INFO] WARNING: No value received for: 'SelectedGrating' yet, sending empty object anyway
[2018-07-11 14:47:33] [INFO] WARNING: No value received for: 'DetailedState' yet, sending empty object anyway


0
1
101.0
0
0


Need to create a command sender (defines a class per commandable entity)

In [3]:
sender = salpylib.DDSSend("atMonochromator")
# sender.start()

[2018-07-11 17:38:10] [DEBUG] Loading Device: atMonochromator


Can disable then re-enable each component to update the state (and publish the events that cause issues below)

In [8]:
cmd_id = sender.send_Command('disable', disable=0, wait_command=False)
print(cmd_id)

[2018-07-11 17:39:24] [DEBUG] Updating myData object with kwargs
[2018-07-11 17:39:24] [DEBUG] disable = 0
[2018-07-11 17:39:24] [DEBUG] Issuing command: disable


(1189641422, 303)


In [17]:
print(rejected.getEvent)

<bound method PyCapsule.getEvent_RejectedCommand of <SALPY_atMonochromator.SAL_atMonochromator object at 0x7f38d730d4c8>>


In [11]:
cmd_id = sender.send_Command('enable', enable=0,wait_command=False)
print(cmd_id)

[2018-07-11 17:39:58] [DEBUG] Updating myData object with kwargs
[2018-07-11 17:39:58] [DEBUG] enable = 0
[2018-07-11 17:39:58] [DEBUG] Issuing command: enable


(596516651, -301)


In [10]:
cmd_id = sender.send_Command('standby', standby=True, wait_command=False)
print(cmd_id)

[2018-07-11 17:39:43] [DEBUG] Updating myData object with kwargs
[2018-07-11 17:39:43] [DEBUG] key standby not in myData
[2018-07-11 17:39:43] [DEBUG] Issuing command: standby


(1025202362, 303)


In [78]:
cmd_id = sender.monochromator.send_Command('start', settingsToApply='Default', wait_command=False)
print(cmd_id)

[2018-07-09 13:06:58] [INFO] Updating myData object with kwargs
[2018-07-09 13:06:58] [INFO] key start not in myData
[2018-07-09 13:06:58] [INFO] settingsToApply = Default
[2018-07-09 13:06:58] [INFO] Issuing command: start


(294702568, -301)


In [54]:
cmd_id = sender.monochromator.send_Command('SelectGrating', gratingType=1)
start_time = time.time()
#while time.time()-start_time < 60:
    

[2018-07-06 18:46:24] [INFO] Updating myData object with kwargs
[2018-07-06 18:46:24] [INFO] gratingType = 1
[2018-07-06 18:46:24] [INFO] Issuing command: SelectGrating


In [55]:
print(sender.monochromator.cmdId, sender.retval)

1540383426 -301


In [60]:
cmd_id = sender.monochromator.send_Command('updateMonochromatorSetup', gratingType=2, 
                             fontExitSlitWidth=1.05, 
                             fontEntranceSlitWidth=2.75,
                             wavelength=400)
print(cmd_id)
start_time = time.time()
#while True:
#    if time.time() - start_time > 120:
#        print('Timed out')
#        break
#    elif detailed_state.getCurrent().detailedState == 7:
#        print('Finished...')
#        break
#    elif ((time.time() - start_time) % 10) == 0:
#        print('waiting...')

[2018-07-06 18:58:22] [INFO] Updating myData object with kwargs
[2018-07-06 18:58:22] [INFO] gratingType = 2
[2018-07-06 18:58:22] [INFO] fontExitSlitWidth = 1.05
[2018-07-06 18:58:22] [INFO] fontEntranceSlitWidth = 2.75
[2018-07-06 18:58:22] [INFO] wavelength = 400
[2018-07-06 18:58:22] [INFO] Issuing command: updateMonochromatorSetup


(304089173, 303)


In [61]:
print(cmd_id)

(304089173, 303)


In [43]:
cmd_id = sender.send_Command('ChangeSlitWidth', slit=1, slitWidth=1.00)
start_time = time.time()
wait_time = 0
while True:
    if time.time() - start_time > 120:
        print('Timed out')
        break
    elif detailed_state.getCurrent().detailedState == 7:
        print('Finished...')
        wait_time = time.time() - start_time
        break
    elif ((time.time() - start_time) % 10) == 0:
        print('waiting...')
print('Waited {} seconds'.format(wait_time))
print(cmd_id)

[2018-07-03 11:31:24] [INFO] Updating myData object with kwargs
[2018-07-03 11:31:24] [INFO] slit = 1
[2018-07-03 11:31:24] [INFO] slitWidth = 1.0
[2018-07-03 11:31:24] [INFO] Issuing command: ChangeSlitWidth


Finished...
Waited 0.0005669593811035156 seconds
(783368695, 303)


In [38]:
print(cmd_id)

(783368693, -301)


BELOW THIS LINE IS TIAGO MESSING AROUND.... No Im not.... 

In [59]:
cmd_id = sender.send_Command('ChangeWavelength', wavelength=420)
start_time = time.time()
wait_time = 0
while True:
    if time.time() - start_time > 120:
        print('Timed out')
        break
    elif detailed_state.getCurrent().detailedState == 7:
        print('Finished...')
        wait_time = time.time() - start_time
        break
    elif ((time.time() - start_time) % 10) == 0:
        print('waiting...')
print('Waited {} seconds'.format(wait_time))
print(cmd_id)

[2018-07-06 18:50:11] [INFO] Updating myData object with kwargs
[2018-07-06 18:50:11] [INFO] wavelength = 420
[2018-07-06 18:50:11] [INFO] Issuing command: ChangeWavelength


Finished...
Waited 0.0006232261657714844 seconds
(1303455738, 303)


In [27]:
detailed_state.start()

In [33]:

print(detailed_state.getCurrent().detailedState)

7


In [8]:
# sender.myData.disable
# sender.cmd
sender.issueCommand(sender.myData)

1025202363

In [17]:
getattr(salpylib.SALPY_lib,'SAL_{}'.format('atMonochromator'))()

AttributeError: module 'salpytools.salpylib' has no attribute 'SALPY_lib'

In [9]:
import SALPY_atMonochromator

In [19]:
import inspect

In [20]:
members = inspect.getmembers(SALPY_atMonochromator)

In [18]:
# inspect.getmembers(sender.SALPY_lib)

In [28]:
for member in members:
    if 'command' in member[0]:
#         print(member)
        cmd_name = member[0].split('command_')[-1][:-1]
        print(cmd_name)

CalibrateWavelength
ChangeLightIntensity
ChangeSlitWidth
ChangeWavelength
Power
PowerWhiteLight
SelectGrating
SetCoolingTemperature
disable
enable
enterControl
exitControl
standby
start
updateMonochromatorSetup


In [11]:
topic.enable = True

In [17]:
manager = SALPY_atMonochromator.SAL_atMonochromator() #.salProcessor('atMonochromator_command_enable')

In [24]:
sender.waitForCompletion['disable']

<bound method PyCapsule.waitForCompletion_disable of <SALPY_atMonochromator.SAL_atMonochromator object at 0x7fac84498260>>